In [1]:
import sys 
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

In [2]:
'''This notebook generates the plots for experiment 1.'''


T = 250
dt = 0.1
t_vec = np.arange(0,T,dt)

'''model params''' 
model_params = {'gamma':1/1000,'L':0.,'D':1/7,'hosp':1/10,'R':1/100,'sig_state':0.005,'lam':1/35,'mean_ou':-1.3,'sig':0.4}

param_names = ['hosp','R','mean_ou','sig','lam']

for run in range(1,51):
    burn_in = 80_000
    output = np.load(f'Results/PMCMC_Output_{run}.npz')
    
    with PdfPages(f'figures/figure_run_{run}.pdf') as pdf:

        fig,axs = plt.subplots(nrows = 2,ncols = 4,figsize = (20,10))

        
        for i in range(3):
            axs[0,i].hist(output['accepted_params'][i,burn_in:],bins = 50)
            axs[0,i].set_title(f"{param_names[i]}, Mean: {np.round(np.mean(output['accepted_params'][i,burn_in:]),2)}, std: {np.round(np.std(output['accepted_params'][i,burn_in:]),2)}, Error: {np.round(np.abs(model_params[param_names[i]] - np.mean(output['accepted_params'][i,burn_in:])),2)}")
            axs[0,i].axvline(x = model_params[param_names[i]],color = 'red')

        for i in range(2):
            axs[1,i].set_title(f"{param_names[i+3]}, Mean: {np.round(np.mean(output['accepted_params'][i+3,burn_in:]),2)}, std: {np.round(np.std(output['accepted_params'][i+3,burn_in:]),2)}, Error: {np.round(np.abs(model_params[param_names[i+3]] - np.mean(output['accepted_params'][i+3,burn_in:])),2)}")
            axs[1,i].hist(output['accepted_params'][i+3,burn_in:],bins = 50)

            axs[1,i].axvline(x = model_params[param_names[i+3]],color = 'red')
            axs[1,i].axvline(x = model_params[param_names[i+3]],color = 'red') 


        axs[0,3].set_title('Simulated $\\beta_t$')
        axs[0,3].fill_between(t_vec[::int(1/dt)],np.percentile(output['MLE_particle_dist'][:, 4, :].T, 12.5, axis=1),
                              np.percentile(output['MLE_particle_dist'][:, 4, :].T, 87.5, axis=1),
                              alpha=0.5, color='steelblue')
        
        axs[0, 3].plot(t_vec, output['betas'], '--', color='black')

        axs[1,2].set_title('Log Likelihood')
        axs[1,2].plot(output['Log_Likelihood'][burn_in:])

        axs[1,3].set_title('Simulated Data')
        axs[1,3].fill_between(t_vec[::int(1/dt)],
                                      np.percentile(output['MLE_Observation_dist'][:, 0, :].T, 12.5, axis=1),
                                      np.percentile(output['MLE_Observation_dist'][:, 0, :].T, 87.5, axis=1),
                                      alpha=0.5, color='steelblue')
        
        axs[1,3].plot(output['data'].T,'--',color = 'black')

        # Save the current figure to the PDF
        pdf.savefig(fig)  # Save the figure to the PDF
        plt.close(fig)    # Close the figure to free up memory


: 